# Работа с Excel

Материалы:
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
import pandas as pd
import openpyxl
import xlwings as xw

In [ ]:
reviews = pd.read_csv("../Pandas/data/reviews_sample.csv", index_col=0)
recipes = pd.read_csv("data/recipes_sample.csv")

recipes.drop('n_steps', axis=1, inplace=True)

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц.

In [ ]:
# Count rows equal to 5 percents of DF
cnt_recipes = round(len(recipes) / 100 * 5)
cnt_reviews = round(len(reviews) / 100 * 5)

# New DF's
part_recipes = recipes.sample(cnt_recipes)
part_reviews = reviews.sample(cnt_reviews)

In [ ]:
with pd.ExcelWriter("data/recipes.xlsx") as writer:
    part_recipes.to_excel(writer, sheet_name="Рецепты")
    part_reviews.to_excel(writer, sheet_name="Отзывы")

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
book = xw.Book('data/recipes.xlsx')
page = book.sheets['Рецепты']

# get count of seconds in each recipe
seconds = part_recipes['minutes'].apply(lambda x: x * 60).tolist()

page.range('I1').options(transpose=True).value = ['seconds_assign'] + seconds

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
page.range('J1').value = 'seconds_formula'
page.range(f'J2:J{len(seconds) + 1}').options(transpose=True).value = "=D2 * 60"

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
page.range('I1:J1').font.bold = True
page.range('I1:J1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
green = (0, 128, 0)
yellow = (255, 255, 0)
red = (255, 0, 0)

for i in range(2, len(seconds) + 2):
    cell = page.range(f'D{i}')
    if 0 <= cell.value <= 5:
        color = green
    elif 5 < cell.value < 10:
        color = yellow
    elif cell.value >= 10:
        color = red
    else:
        raise ValueError("Unacceptable value for time")
    cell.color = color

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
page.range('K1').value = 'n_reviews'
page.range(f'K2:K{len(seconds) + 1}').options(transpose=True).formula = "=COUNTIF(Отзывы!$C:$C,C2)"

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`

В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
# ДОЛГО, но что поделать...

# hash-table for fast searching id on sheet 'Рецепты'
recipes_dct = set(part_recipes['id'].tolist())

last_col = book.sheets[1]['A1'].expand("right").last_cell.address.split("$")[1]


def validate(row_num: int):
    # Check value in [0, 5]
    rate_validation = book.sheets[1].range(f"E{row_num}").value in range(6)
    include_validation = book.sheets[1].range(f'C{row_num}').value in recipes_dct
    if not (rate_validation and include_validation):
        book.sheets[1].range(f'A{row_num}:{last_col}{row_num}').color = red


for review in range(2, book.sheets[1].range(f'E2').end('down').row):
    validate(review)

In [ ]:
book.save()
book.close()

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [26]:
# Спасибо за cp1251)

import csv

with open('data/recipes_model.csv', 'r') as csv_file:
    reader = csv.reader(csv_file, delimiter='\t')
    headers = next(reader)
    model = []
    for i in reader:
        row = list(map(lambda x: x.encode('cp1251').decode('utf8'), i))
        model.append(row)

In [38]:
model_book = xw.Book()
model_book.sheets[0].name = 'Модель'
model_page = model_book.sheets[0]
model_page.range('A1').value = ['A', 'B', 'C', 'Обязательно к заполнению', 'E', 'F', 'Ключ', 'H', 'I']
model_page.range('A2').value = model
model_book.save('data/recipes_model.xlsx')
model_book.close()

![image.png](https://i.postimg.cc/qqpkGzSL/image.png)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом

    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"

    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [39]:
# Подключаем add-in xlwings для работы sql запроса без лишних манипуляций с VBA
xw.Book("C:\\Users\\slava\\AppData\\Roaming\\Microsoft\\Excel\\XLSTART\\xlwings.xlam")
book = xw.Book('data/recipes_model.xlsx')
model_page = book.sheets['Модель']

sql_formula = "SELECT a.B || ' ' || UPPER(a.C) || ' ' || " \
              "CASE WHEN a.Ключ = 'PK' THEN 'PRIMARY KEY' WHEN a.Ключ = 'FK' THEN 'REFERENCES' || ' ' " \
              "|| COALESCE(a.H, ' ') || ' (' || COALESCE(a.I, ' ') || ') ' ELSE ' ' END || " \
              "CASE WHEN [Обязательно к заполнению] = 'Y' AND (a.Ключ <> 'PK' OR a.Ключ IS NULL) " \
              "THEN 'NOT NULL' ELSE ' ' END from a"

table = model_page.range('A1').expand('table')
table_address = table.address

last_col = table_address.split(':')[1].split("$")[1]
last_row = table.end('down').row

# Без этого костыля.... НИКАК
formula_cell = f'K{last_row + 5}'
model_page.range(f'{formula_cell}').value = sql_formula

model_page.range('J1').formula_array = f"""=sql({formula_cell}; {table_address})"""

![image.png](https://i.postimg.cc/MpShMCk7/image.png)

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [46]:
headers = model_page.range(f'A1:{last_col}1') 
headers.color = '#00ccff'
headers.font.bold = True
# Костыль с формулой всё делает не очень приятно, как пофиксить 0 идей
model_page.autofit(axis='columns')

model_page.tables.add(table, name="SQL")
model_page.tables["SQL"].show_autofilter = True

In [36]:
book.save()
book.close()

![image.png](https://i.postimg.cc/g2cbtCMY/image.png)

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) #%% md

In [ ]:
#Какие атрибуты, какие сущности...? 